In [1]:
import warnings
import numpy as np
import pandas as pd
from pathlib import Path
import os
import vectorbt as vbt
import io
import sys
from contextlib import redirect_stdout
from datetime import datetime
import math
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import warnings
warnings.filterwarnings('ignore')

# Add project root to path
project_root = Path().absolute().parent
sys.path.append(str(project_root))

# Import utils with different aliases
from src.utils import csv_exporter as csv_utils
from src.utils import validation as val_utils
from src.utils import transformations as trans_utils
from src.utils import data_merger as merge_utils
from src.utils import config_validator as config_utils
from src.utils import metrics as metric_utils
from src.core.bloomberg_fetcher import fetch_bloomberg_data
from src.utils.transformations import get_ohlc



In [7]:
# Getting all the data 
mapping = {
    ('I05510CA Index', 'INDEX_OAS_TSY_BP'): 'cad_oas',
    ('LF98TRUU Index', 'INDEX_OAS_TSY_BP'): 'us_hy_oas',
    ('LUACTRUU Index', 'INDEX_OAS_TSY_BP'): 'us_ig_oas',
    ('SPTSX Index', 'PX_LAST'): 'tsx',
    ('VIX Index', 'PX_LAST'): 'vix',
}

# Calculate dates
end_date = datetime.now().strftime('%Y-%m-%d')
start_date ='2002-01-01'

# Fetch the data
df = fetch_bloomberg_data(
    mapping=mapping,
    start_date=start_date,
    end_date=end_date,
    periodicity='D',
    align_start=True
).dropna()

# Getting all the er_ytd data 
mapping1 = {
    ('I05510CA Index', 'INDEX_EXCESS_RETURN_YTD'): 'cad_ig_er',
    ('LF98TRUU Index', 'INDEX_EXCESS_RETURN_YTD'): 'us_hy_er',
    ('LUACTRUU Index', 'INDEX_EXCESS_RETURN_YTD'): 'us_ig_er',
}

# Fetch the er_ytd_data
df1 = fetch_bloomberg_data(
    mapping=mapping1,
    start_date=start_date,
    end_date=end_date,
    periodicity='D',
    align_start=True
).dropna()

# Convert er_ytd data to an index
df2 = trans_utils.convert_er_ytd_to_index(df1[['cad_ig_er','us_hy_er','us_ig_er']])
final_df = merge_utils.merge_dfs(df, df2, fill='ffill', start_date_align='yes')

# Handle bad data point for cad_oas on Nov 15 2005
bad_date = '2005-11-15'
if bad_date in final_df.index:
    final_df.loc[bad_date, 'cad_oas'] = final_df.loc[final_df.index < bad_date, 'cad_oas'].iloc[-1]

def calculate_qtd_changes(df):
    df_copy = df.copy()
    df_copy.index = pd.to_datetime(df_copy.index)
    
    qtd_changes = pd.DataFrame(index=df_copy.index)
    for col in df_copy.columns:
        changes = []
        for date in df_copy.index:
            # Get the start of the current quarter
            current_quarter_start = date.to_period('Q').start_time
            
            # Get the end of previous quarter (last day of previous quarter)
            prev_quarter_end = current_quarter_start - pd.Timedelta(days=1)
            
            # Get the value at the end of previous quarter
            mask = df_copy.index <= prev_quarter_end
            if not mask.any():
                changes.append(np.nan)
                continue
            
            prev_quarter_value = df_copy[col][mask].iloc[-1]
            current_value = df_copy[col][date]
            
            # Calculate change
            pct_change = (current_value / prev_quarter_value - 1) * 100
            changes.append(pct_change)
            
        qtd_changes[f'{col}_qtd'] = changes
    return qtd_changes

def calculate_ytd_changes(df):
    df_copy = df.copy()
    df_copy.index = pd.to_datetime(df_copy.index)
    
    ytd_changes = pd.DataFrame(index=df_copy.index)
    for col in df_copy.columns:
        changes = []
        for date in df_copy.index:
            # Get the start of the current year
            current_year_start = date.to_period('Y').start_time
            
            # Get the end of previous year (last day of previous year)
            prev_year_end = current_year_start - pd.Timedelta(days=1)
            
            # Get the value at the end of previous year
            mask = df_copy.index <= prev_year_end
            if not mask.any():
                changes.append(np.nan)
                continue
            
            prev_year_value = df_copy[col][mask].iloc[-1]
            current_value = df_copy[col][date]
            
            # Calculate change
            pct_change = (current_value / prev_year_value - 1) * 100
            changes.append(pct_change)
            
        ytd_changes[f'{col}_ytd'] = changes
    return ytd_changes

def add_all_changes(df):
    # Make a copy to avoid modifying original
    result_df = df.copy()
    result_df.index = pd.to_datetime(result_df.index)
    
    # Calculate regular percentage changes (3M and 1YR)
    for col in df.columns:
        # 3-month change (63 trading days approximation)
        result_df[f'{col}_3m'] = df[col].pct_change(periods=63) * 100
        # 1-year change (252 trading days approximation)
        result_df[f'{col}_1yr'] = df[col].pct_change(periods=252) * 100
    
    # Add QTD and YTD changes
    qtd_df = calculate_qtd_changes(df)
    ytd_df = calculate_ytd_changes(df)
    
    # Combine all changes
    for col in qtd_df.columns:
        result_df[col] = qtd_df[col]
    for col in ytd_df.columns:
        result_df[col] = ytd_df[col]
    
    # Drop any rows with missing values
    result_df = result_df.dropna()
    
    return result_df

def verify_calculations(df, n_random_samples=5):
    """Comprehensive verification of calculations"""
    print("Running verification checks...")
    print("\n1. Random Date Samples:")
    print("=" * 80)
    
    # Get random dates
    random_dates = df.sample(n=n_random_samples).index.sort_values()
    
    for date in random_dates:
        date = pd.to_datetime(date)
        
        # Get period boundaries
        current_quarter_start = date.to_period('Q').start_time
        current_year_start = date.to_period('Y').start_time
        prev_quarter_end = current_quarter_start - pd.Timedelta(days=1)
        prev_year_end = current_year_start - pd.Timedelta(days=1)
        
        print(f"\nDate: {date.strftime('%Y-%m-%d')}")
        print(f"Previous Quarter End: {prev_quarter_end.strftime('%Y-%m-%d')}")
        print(f"Previous Year End: {prev_year_end.strftime('%Y-%m-%d')}")
        
        # Check calculations for TSX
        col = 'tsx'
        current_value = df[col][date]
        prev_quarter_value = df[df.index <= prev_quarter_end][col].iloc[-1]
        prev_year_value = df[df.index <= prev_year_end][col].iloc[-1]
        
        manual_qtd = (current_value / prev_quarter_value - 1) * 100
        manual_ytd = (current_value / prev_year_value - 1) * 100
        
        print(f"\nTSX Values:")
        print(f"Current: {current_value:.2f}")
        print(f"Prev Quarter End: {prev_quarter_value:.2f}")
        print(f"Prev Year End: {prev_year_value:.2f}")
        print(f"QTD% (Manual): {manual_qtd:.2f}%")
        print(f"QTD% (Calculated): {df[f'{col}_qtd'][date]:.2f}%")
        print(f"YTD% (Manual): {manual_ytd:.2f}%")
        print(f"YTD% (Calculated): {df[f'{col}_ytd'][date]:.2f}%")
        
        # Check for significant differences
        qtd_diff = abs(manual_qtd - df[f'{col}_qtd'][date])
        ytd_diff = abs(manual_ytd - df[f'{col}_ytd'][date])
        
        if qtd_diff > 0.01:
            print(f"WARNING: Large QTD difference ({qtd_diff:.4f}%)")
        if ytd_diff > 0.01:
            print(f"WARNING: Large YTD difference ({ytd_diff:.4f}%)")
            
    print("\n2. Quarter/Year Boundary Checks:")
    print("=" * 80)
    
    # Find dates near quarter/year boundaries
    quarter_starts = df.index[pd.to_datetime(df.index).is_quarter_start]
    year_starts = df.index[pd.to_datetime(df.index).is_year_start]
    
    if len(quarter_starts) > 0:
        print("\nFirst day of quarter example:")
        first_quarter_date = quarter_starts[len(quarter_starts)//2]  # Take a middle example
        print(f"Date: {first_quarter_date}")
        print(f"TSX QTD%: {df.loc[first_quarter_date, 'tsx_qtd']:.2f}%")
        
    if len(year_starts) > 0:
        print("\nFirst day of year example:")
        first_year_date = year_starts[len(year_starts)//2]  # Take a middle example
        print(f"Date: {first_year_date}")
        print(f"TSX YTD%: {df.loc[first_year_date, 'tsx_ytd']:.2f}%")

# Calculate all changes
print("Calculating changes...")
final_df = add_all_changes(final_df)

# Run comprehensive verification
print("\nRunning verification...")
verify_calculations(final_df)

# Show basic info about the final dataset
print("\nFinal Dataset Info:")
print("=" * 80)
final_df.info()

Calculating changes...

Running verification...
Running verification checks...

1. Random Date Samples:

Date: 2010-09-03
Previous Quarter End: 2010-06-30
Previous Year End: 2009-12-31

TSX Values:
Current: 12144.92
Prev Quarter End: 11294.42
Prev Year End: 11746.11
QTD% (Manual): 7.53%
QTD% (Calculated): 7.53%
YTD% (Manual): 3.40%
YTD% (Calculated): 3.40%

Date: 2013-11-20
Previous Quarter End: 2013-09-30
Previous Year End: 2012-12-31

TSX Values:
Current: 13430.01
Prev Quarter End: 12787.19
Prev Year End: 12433.53
QTD% (Manual): 5.03%
QTD% (Calculated): 5.03%
YTD% (Manual): 8.01%
YTD% (Calculated): 8.01%

Date: 2016-07-13
Previous Quarter End: 2016-06-30
Previous Year End: 2015-12-31

TSX Values:
Current: 14493.80
Prev Quarter End: 14064.54
Prev Year End: 13009.95
QTD% (Manual): 3.05%
QTD% (Calculated): 3.05%
YTD% (Manual): 11.41%
YTD% (Calculated): 11.41%

Date: 2017-02-08
Previous Quarter End: 2016-12-31
Previous Year End: 2016-12-31

TSX Values:
Current: 15554.04
Prev Quarter End: